In [ ]:
import os
def check_if_experiment_already_run(ALPHA, APPLY_EVERY_N, SKIP_FIRST_N, EXP_TARGET):
  LOG_FILENAME = f"SKIP_{SKIP_FIRST_N}_ALPHA_{ALPHA}-APPLY_EVERY_N_{APPLY_EVERY_N}-log.txt"
  LOG_PATH = os.path.join(f"record/activation_shaping_experiments/{EXP_TARGET}", LOG_FILENAME)
  return os.path.exists(LOG_PATH)

def run_experiment(category, alphas, apply_every_n_list, skip_first_n_list):
  for alpha in alphas:
    for apply_every_n in apply_every_n_list:
      for skip_first_n in skip_first_n_list:
        if (not check_if_experiment_already_run(alpha, apply_every_n, skip_first_n, category)):
          !launch_scripts/activation_shaping_experiments.sh {category} "" "{alpha}" "{apply_every_n}" "{skip_first_n}"
        else:
          print("Already existing")

          

alphas = [1, 0.9, 0.8, 0.6, 0.4]
apply_every_n_list = [0, 1, 2]
skip_first_n_list = [15, 14, 13, 10, 4 ]
run_experiment("photo,cartoon,sketch", alphas, apply_every_n_list, skip_first_n_list)

In [ ]:
!./launch_scripts/domain_adaptation.sh "cartoon" "" 0 2 14 "threshold"
!./launch_scripts/domain_adaptation.sh "cartoon" "" 0.2 2 14 "topk"